## Linear models


We assume that the points follow a linear model
$$
\begin{bmatrix}
y_1  \\
\vdots \\
y_n 
\end{bmatrix}_{nx1} = 
\begin{bmatrix}
1 & X_{11} & X_{21} & \dots & X_{k1}  \\
1 &  X_{12} & X_{22} & \dots & X_{k1} \\
\vdots &  \vdots & \vdots & \vdots & \vdots \\
1 &  X_{1n} & X_{2n} & \dots & X_{kn} \\
\end{bmatrix}_{nxk} \begin{bmatrix}
\beta_1  \\
\vdots \\
\beta_k 
\end{bmatrix}_{kx1}
+
\begin{bmatrix}
\varepsilon_1  \\
\vdots \\
\varepsilon_k 
\end{bmatrix}_{nx1}
$$

$$
    y=X\beta + \varepsilon
$$

The following are a set of methods intended for regression in which the target value is expected to be a linear combination of the features. In mathematical notation, if 
 is the predicted value.

$$
\hat{y}(\beta, x) = \beta_0 + \beta_1 x_1 + ... + \beta_p x_p
$$

Across the module, we designate the vector $\beta = (\beta_1,..., \beta_p)$ as `coef_` and $\beta_0$ as `intercept_`

> partial source https://scikit-learn.org/stable/modules/linear_model.html


Solving

$$
\min_{w} || X w - y||_2^2
$$
Yields the estimate frfom last time
$$
\beta = (X^TX)^{-1}X^Ty
$$

### From last time

In [58]:
import plotly.graph_objects as go
import numpy as np
n = 200
np.random.seed(seed=1)
# x co-ordinates
x = np.arange(0, n)/100
X = np.array([x, np.ones(n)])

y = x*10 + np.random.normal(0,3,n)

In [59]:
Xt = X.T
inv = np.linalg.inv(Xt.T@Xt)
xy = Xt.T@y
beta = inv@xy

line = beta[0]*x + beta[1] 

### Can be solved using the linea_model module from `sklearn`

In [65]:
from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit(X[0,:].reshape(-1, 1),y)

fig = go.Figure()

line1 = reg.coef_[0]*x + reg.intercept_
# Add traces
fig.add_trace(go.Scatter(x=x, y=y,mode='markers',name='observations'))
fig.add_trace(go.Scatter(x=x, y=line,mode='lines',name='OLS formula'))
fig.add_trace(go.Scatter(x=x, y=line1,mode='lines',name='Sklearn'))

### Mini exerceise

- Fit en model for avocado pris-sættet
- Modellen skal lade AveragePrice som funktion af Total volume
- Undersøg koefficienter
- Plot evt. regressions-linjen

```
Coef = pd.DataFrame([reg.coef_,df[predictors_1].columns],index=["Beta","Predictor"]).T
Coef.sort_values("Beta")
```

$$
    \text{AveragePrice} = \alpha + \beta \cdot \text{Total Volume} + \varepsilon, \quad \varepsilon \sim N(0,1)
$$

### Going to higher dimensions


In [8]:
import pandas as pd
import plotly.express as px

In [9]:
yield_huf = pd.read_excel('../data/xl/R.xlsx',header = 0)
yield_huf.Date = pd.to_datetime(yield_huf.Date, unit="D")
yield_huf.loc[:,"lasso_predictions"] = 0
yield_huf.loc[:,"predictions"] = 0
yield_huf.head(3)

,CDS,CPI,FX_vol,Growth_indicator,Forwards,CA,M2,Trade,Reserves,IP_Price,...,Aggreculture,Copper,Treasury,Yield,1y10y,Treasuries_Forecast,NDF,Date,lasso_predictions,predictions
0,246.345,6.4,10.92,52.5,6.7427,-0.165,2.564835,5.124248,13928,188.1033,...,327.824,6745.0,3.196,7.689,3.6795,3.80,282.4083,2010-01-30,0,0
1,237.673,5.7,10.44,54.9,6.4879,-0.165,0.933108,5.124248,13185,184.5910,...,332.378,7195.0,3.101,7.555,3.6097,3.90,281.1995,2010-02-27,0,0
2,182.833,5.9,8.58,52.8,5.5355,-0.165,-0.729452,5.124248,8528,171.0417,...,293.690,7790.0,3.092,6.884,3.5956,3.39,275.4619,2010-04-01,0,0


In [10]:
yield_huf["lag_1"] = yield_huf.Yield.shift(1,fill_value=7)
yield_huf["lag_2"] = yield_huf.Yield.shift(2,fill_value=7)
yield_huf["moving_average"] = (yield_huf.lag_1 + yield_huf.lag_2)/2 

In [11]:
yield_huf_melt = yield_huf.melt(id_vars = "Date")
yield_huf_melt.head()

,Date,variable,value
0,2010-01-30,CDS,246.345
1,2010-02-27,CDS,237.673
2,2010-04-01,CDS,182.833
3,2010-05-01,CDS,195.482
4,2010-06-01,CDS,246.735


In [12]:
target = "Yield"
px.line(
    yield_huf_melt[yield_huf_melt['variable'].isin([target])],
    x="Date", 
    y = "value",
    title="Hungarian 10 year bond yield",
    labels={"value":"Yield in pct"}
)

In [13]:
predictors = ["CPI", "M2", "Forwards","FX_vol","CA","Trade"]
px.line(
    yield_huf_melt[yield_huf_melt['variable'].isin(predictors)],
    x="Date", 
    y = "value",  
    facet_col="variable",
    color="variable",
    title="Macro-economic variables for Hungary"
)

### `sklearn` accepts both pandas and numpy objects

General pattern is 

```python
from sklearn.x import Model

model = Model()
model.fit(X,y)
prediction = model.predict(X)
```

To get `numpy` arrays from a pandas object, simply call their `values` property

```python
df = pd.DataFrame(data)

predictor_list = ["A","B","C"]
target = "D"

X,y = df[predictor_list].values, df[target].values
``` 

For out hungarien yield rates this look like:

$$
    \text{yield}_t = \alpha + \beta_1 \text{CDS}_t + \beta_2\text{M2}_t + \beta_3 \text{CPI}_t
$$

In [14]:
X,y = yield_huf[predictors].values, yield_huf[target].values
X.shape

(87, 6)

## Note the OOP nature of models in `sklearn`

The state of the model is preserved within properties of the instantiated class object

```
model = Model()
model.fit(X,y)
prediction = model.predict(X)

model.coef_
model.copy_X
model.tol
```


In [15]:
reg = linear_model.LinearRegression()
# Could have used  reg.fit(X,y)
reg.fit(yield_huf[predictors], yield_huf[target])
gamma = reg.coef_

# If we use numpy objects we can use:
#
# yield_huf.loc[:,"pred"] = reg.intercept_ + X@gamma

yield_huf.loc[:,"predictions"] = reg.predict(yield_huf[predictors])

# Add traces
yield_huf_melt = pd.melt(yield_huf[["Date","Yield","predictions"]], id_vars='Date',value_name="value")
px.line(yield_huf_melt, x="Date",y = "value", line_dash= "variable", color = "variable")

## Quite solid regression, however..

Hvis vi lader modellen fittes til observationer før 2013-01-01, og predikterer på baggrund af observerede værdier sidenhen ses svagheden



In [16]:
def lines_with_shape(
    df,
    cols =[{"x":"Date","y":"Yield"}], 
    names=["Yield"], 
    shape_dates=["2010-01-01","2013-01-01"],
    title="Splitting the data set into a 'training set' and a 'test set'"
):

    fig = go.Figure()
    offset = 0.35
    for line_cols, line_name in zip(cols,names):

        fig.add_trace(go.Scatter(
        x=df[line_cols["x"]],
        y=df[line_cols["y"]],
        mode="lines",
        name=line_name,
        ))

    # Add shape regions
    fig.update_layout(
        shapes=[
            go.layout.Shape(
                type="rect",
                xref="x",
                yref="y",
                x0=pd.to_datetime(shape_dates[0]),
                y0=df[cols[0]["y"]].min()-offset,
                x1=pd.to_datetime(shape_dates[1]),
                y1=df[cols[0]["y"]].max()+offset,
                fillcolor="LightGreen",
                opacity=0.3,
                line_width=0,
            )],
        title=title
    )
    return fig

In [17]:
lines_with_shape(yield_huf)

In [18]:
split_date = "'2013-01-01'"
train = yield_huf.query(f'Date <= {split_date}')
test = yield_huf.query(f'Date > {split_date}')

reg = linear_model.LinearRegression()
reg.fit(train[predictors], train[target])

yield_huf.loc[:,"predictions"] = reg.predict(yield_huf[predictors])

In [19]:
lines_with_shape(yield_huf,
                 cols=[{"x":"Date","y":"Yield"},{"x":"Date","y":"predictions"}],
                 names=["Yield","OLS predictions"],
                 title="Out of sample predtion with OLS")

### Problem persists even when more features are included

In [20]:
predictors_1 = list(set(yield_huf.columns) - set([target,"Date","predictions","lasso_predictions","lag_1","lag_2"]))

In [21]:
reg = linear_model.LinearRegression()
reg.fit(train[predictors_1], train[target])
yield_huf.loc[:,"predictions"] = reg.predict(yield_huf[predictors_1])

In [23]:
lines_with_shape(yield_huf,
                 cols=[{"x":"Date","y":"Yield"},{"x":"Date","y":"predictions"}],
                 names=["Yield","OLS predictions"],
                 title="Out of sample predtion with OLS")

### Issue is multi-colinearity

In [25]:
Coef = pd.DataFrame([reg.coef_,yield_huf[predictors_1].columns],index=["Beta","Predictor"]).T
Coef.sort_values("Beta")

,Beta,Predictor
16,-2.91312,Treasury
0,-0.52387,moving_average
11,-0.417379,Treasuries_Forecast
8,-0.0313073,Trade
4,-0.0138744,NDF
9,-0.0138633,IP_Price
3,-0.00679222,Aggreculture
17,-0.00154664,Energy
19,4.31925e-05,Reserves
12,0.000116539,Platinium


## LASSO

Problemet i ovenstående regression kan være at koefficienterne når uhensigtsmæssigt høje værdier grundet multi kolinaritet. Dette kan føre til et fit som genneraliserer dårligt out-of sample. En måde at modvirke dette er at begrænse flekskibiliteten for en lineær regression ved eksempel vis en $ \mathcal{L}_1 $ straf som i LASSO (Least Absolute Shrinkage and Selection Operator)

$$
    \hat{\beta} = \min_\beta (y-X\beta)^T (y-X\beta) + \lambda\sum_j |\beta_j|, \quad \lambda\geq 0
$$

In [26]:
res = []
grid = np.linspace(0.01, 5, 50)
for alpha in grid:
    reg = linear_model.Lasso(alpha=alpha)
    #reg.fit(X,y)
    #res.append(reg.predict(X))
    reg.fit(yield_huf[predictors], yield_huf[target])
    res.append(reg.predict(yield_huf[predictors]))

In [27]:
df = pd.DataFrame(res, index = [str(x) for x in grid]).T
df.loc[:,"Date"] = yield_huf["Date"]
df = pd.melt(df, id_vars=["Date"])
df.loc[:,"variable"] = df.loc[:,"variable"].astype("float")
df.head()

,Date,variable,value
0,2010-01-30,0.01,7.982989
1,2010-02-27,0.01,7.720890
2,2010-04-01,0.01,6.869943
3,2010-05-01,0.01,6.838473
4,2010-06-01,0.01,6.819581


### Illustration of flexibility

- The color scale is the level of penalty added to large coefficients $\alpha$
- Note that when $\alpha$ gets large the predictions becomes the `in-sample mean` of the target



In [28]:
px.scatter(
    df, 
    x="Date",
    y = "value", 
    color = "variable", 
    color_continuous_scale=px.colors.sequential.Viridis)

### Forskellen på de to regressions former ses ved prediction out of sample

In [29]:
reg = linear_model.LassoLars(alpha=.01)
reg.fit(train[predictors_1], train[target])
yield_huf.loc[:,"lasso_predictions"] = reg.predict(yield_huf[predictors_1])

lines_with_shape(yield_huf,
                 cols=[{"x":"Date","y":"Yield"},
                       {"x":"Date","y":"predictions"},
                       {"x":"Date","y":"lasso_predictions"}],
                 names=["Yield","OLS predictions","LASSO predictions"],
                 title="Out of sample comparison of LASSO and OLS")

### Caution with LASSO

LassoLars is a lasso model implemented using the LARS algorithm, and unlike the implementation based on coordinate descent, this yields the exact solution, which is piecewise linear as a function of the norm of its coefficients.



In [30]:
reg = linear_model.Lasso(alpha=0.02,max_iter=100000)
reg.fit(train[predictors_1], train[target])
yield_huf.loc[:,"lasso_predictions_less"] = reg.predict(yield_huf[predictors_1])

In [31]:
lines_with_shape(yield_huf,
                 cols=[{"x":"Date","y":"Yield"},
                       {"x":"Date","y":"predictions"},
                       {"x":"Date","y":"lasso_predictions_less"}],
                 names=["Yield","OLS predictions","LASSO predictions"],
                 title="Out of sample predtion with OLS")

## Mål for error
Forskellige mål for error kan anvendes vi vil kigge på MSE [Mean Square Error] og MAE [Mean Absolute Error], defineret som

$$
L_1 \;   [\text{MAE}] : \frac{1}{T}\sum_t^T |f(x_t)-y_t| \\
L_2 \;   [\text{MSE}] : \frac{1}{T}\sum_t^T (f(x_t)-y_t)² 
$$

Man kan vise at der gælder

$$
L_1 \; : f(x) = \text{median}[Y|X=x] \iff f(x) = \arg\min_c E_{Y|X}[|Y-c||X=x] \\
  L_2 \; : f(x) = E[Y|X=x] \iff f(x) = \arg\min_c E_{Y|X}[(Y-c)²|X=x]
$$

> source p. 18 Elements of statistical learning

### `sklearn.metrics`

The sklearn.metrics module implements several loss, score, and utility functions to measure regression performance. Some of those have been enhanced to handle the multioutput case: mean_squared_error, mean_absolute_error, explained_variance_score and r2_score.

> source https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics

In [32]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error,r2_score

In [33]:
def metrics_table(y_true,y_pred, index="Errors"):
    fs = [explained_variance_score, mean_absolute_error, mean_squared_error,r2_score]
    cols = []
    vals = []
    
    for f in fs:
        vals.append(np.round(f(y_true,y_pred),3))
        cols.append(f.__name__)
    
    return pd.DataFrame([vals], columns=cols,index=[index])

errors=yield_huf.query(f'Date>{split_date}')

pd.concat([metrics_table(errors.Yield,errors.predictions,index="OLS"),
         metrics_table(errors.Yield,errors.lasso_predictions,index="LASSO")])

,explained_variance_score,mean_absolute_error,mean_squared_error,r2_score
OLS,0.049,1.440,3.204,-1.123
LASSO,0.843,0.711,0.710,0.529


## $R^2$ can be negative 

As such variance is dataset dependent, R² may not be meaningfully comparable across different datasets. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R² score of 0.0.

> source https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination

# How to select $\alpha$ used in LASSO?

<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png">

# What is cross validation

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png">

> source: https://scikit-learn.org/stable/modules/cross_validation.html

## K-fold cross validation

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0041.png">

#### Any issue with this method?

#### Time series split

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_0101.png">

### from sklearn.model_selection import TimeSeriesSplit

In [34]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
for train_s, test_s in tscv.split(train.head(10)):
    print("%s %s" % (train_s, test_s))

[0 1 2 3] [4 5]
[0 1 2 3 4 5] [6 7]
[0 1 2 3 4 5 6 7] [8 9]


In [35]:
tscv = TimeSeriesSplit(n_splits=5)
model = linear_model.LassoLarsCV(verbose=True,cv=tscv)
model.fit(train[predictors_1], train[target])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


LassoLarsCV(copy_X=True, cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
            eps=2.220446049250313e-16, fit_intercept=True, max_iter=500,
            max_n_alphas=1000, n_jobs=None, normalize=True, positive=False,
            precompute='auto', verbose=True)

In [36]:
df = pd.DataFrame(
    [
        np.mean(model.mse_path_,axis=1),
        model.cv_alphas_
    ],
    index=["error","alpha"]
).T
px.line(df,x="alpha",y="error")

### General approach

In [37]:
from sklearn.model_selection import cross_val_score

cv = TimeSeriesSplit(n_splits=5)
clf = linear_model.Ridge()

cross_val_score(clf, train[predictors], train[target], cv=cv, scoring='neg_mean_absolute_error')

array([-0.74142506, -0.59390906, -0.82659319, -0.21850187, -0.36913741])

In [38]:
grid = np.linspace(0.5,100,200)
errors = []
for alpha in grid:
    
    clf = linear_model.Ridge(alpha = alpha)
    cv_error = cross_val_score(clf, train[predictors], train[target], cv=cv, scoring='neg_mean_absolute_error')
    errors.append(np.mean(-cv_error))

In [39]:
df = pd.DataFrame([errors,grid],index=["error","alpha"]).T
df
px.line(df,x="alpha",y="error")

### Find valid scoring functions

In [40]:
from sklearn import metrics
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

### Exercise try to improve

Choose k of n models from set


### Exercise hyper-parameter tuning



## Appendix

In [344]:
from tqdm import tqdm
res = []
grid = np.linspace(0.00001, 0.1, 1000)
for alpha in tqdm(grid):
    reg = linear_model.Lasso(alpha=alpha,)
    reg.fit(train[predictors_1], train[target])

    yield_huf.loc[:,"lasso_predictions"] = reg.predict(yield_huf[predictors_1])
#     print(f"alpha= {alpha}")
    error = np.mean(
        np.abs(
            yield_huf.query(f'Date > {split_date}').Yield-
            yield_huf.query(f'Date > {split_date}').lasso_predictions
        )
        )
    res.append(error)
#     print(error)

  0%|          | 0/1000 [00:00<?, ?it/s]/home/christian/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.34511454629549004, tolerance: 0.0020928457638888886

/home/christian/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.36458412491795705, tolerance: 0.0020928457638888886

/home/christian/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.37881047532412127, tolerance: 0.0020928457638888886

/home/christian/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning:

Objective did not

In [345]:
#px.line(x=grid,y=res)
# yield_huf[predictors_1+[target,"Date"]].to_csv("/home/christian/test.csv",index=False)